In [1]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

import import_ipynb
from QBert import train_utils, models

from tqdm.notebook import tqdm


importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\train_utils.ipynb
importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\models.ipynb


## 해당 파일은 bert.run_pretraining.run_bert_pretrain을 구현하는 것을 목표로 한다.

 - Parameter는 FLAG 형식에서 직접 정의해주는 방식으로 변경하고, Main에서 직접 정의하도록 한다.

## Model Test 

In [2]:
filenames = ['./Test_Examples.tfrecords', './Test_Examples.tfrecords']

# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([256], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([256], tf.int64),
    'input_mask': tf.io.FixedLenFeature([256], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([255], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

# keys = feature_description.keys()

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

In [36]:
# sub_model (core_model 필요 Config)

vocab_size = 32000 # 
hidden_size = 768 # Transformer hidden Layers
type_vocab_size = 12 #: The number of types that the 'type_ids' input can take.
num_layers = 12
num_attention_heads = 12
max_seq_length = 256 # 512
dropout_rate = .1
# attention_dropout_rate = .1
inner_dim = 3072
# hidden_act = 'gelu'
initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)

# Pretrain Model 필요 Config
max_predictions_per_seq = 255


In [37]:
def loss_fn(fake_y, losses, **unused_args) :
    
    return tf.reduce_mean(losses, axis = -1)
    

In [38]:
def _select_data_from_record(record):
    """Filter out features to use for pretraining."""
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids'],
        'masked_lm_positions': record['masked_lm_positions'],
        'masked_lm_ids': record['masked_lm_ids'],
        'masked_lm_weights': record['masked_lm_weights'],
    }
    if use_next_sentence_label:
        x['next_sentence_labels'] = record['next_sentence_labels']
    if use_position_id:
        x['position_ids'] = record['position_ids']

    # TODO(hongkuny): Remove the fake labels after migrating bert pretraining.
    if output_fake_labels:
        return (x, record['masked_lm_weights'])
    else:
        return x


In [39]:
train_dataset = tf.data.TFRecordDataset(filenames)
train_dataset = train_dataset.map(_parse_function)

model, sub_model = models.get_bert_models_fn(vocab_size
             , hidden_size
             , type_vocab_size
             , num_layers
             , num_attention_heads
             , max_seq_length
             , max_predictions_per_seq
             , dropout_rate
             , inner_dim 
             , initializer)

In [40]:
use_next_sentence_label = True
output_fake_labels = True
use_position_id = False

dataset = train_dataset.map(_select_data_from_record)

In [41]:
optimizer = tf.keras.optimizers.Adam(lr = 1e-3)

model.compile(optimizer = optimizer, loss=loss_fn)

In [42]:
model.fit(dataset.batch(32), epochs = 1)

C:\Users\LGCNS\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


      8/Unknown - 70s 9s/step - loss: 11.1807 - masked_lm_accuracy: 0.0000e+00 - lm_example_loss: 10.3470 - next_sentence_accuracy: 0.5078 - next_sentence_loss: 0.8337

ResourceExhaustedError:  OOM when allocating tensor with shape[8160,32000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node model_6/bert_pretrain_loss_and_metric_layer_6/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <string>:49) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_118016]

Function call stack:
train_function


## Strategy

In [5]:
tf.config.list_physical_devices() # device 확인

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [6]:
# Strategy 정의

distribution_strategy = 'mirrored' # 'tpu'
num_gpus = 0
all_reduce_alg = None

if distribution_strategy == 'tpu' :
    tpu_address = ""
else :
    tpu_address = None



strategy = train_utils.get_distribution_strategy(
                  distribution_strategy=distribution_strategy,
                  num_gpus=num_gpus,
                  all_reduce_alg=all_reduce_alg,
                  tpu_address=tpu_address)

print ('\n장치의 수: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)

장치의 수: 1


## Input_Files

In [4]:
filenames = ['./Test_Examples.tfrecords', './Test_Examples.tfrecords']

# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([256], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([256], tf.int64),
    'input_mask': tf.io.FixedLenFeature([256], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([255], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

# keys = feature_description.keys()

In [5]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

BUFFER_SIZE = 672

BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

EPOCHS = 10

NameError: name 'strategy' is not defined

In [6]:
train_dataset = tf.data.TFRecordDataset(filenames)

In [7]:
raw_data = next(iter(train_dataset))

In [9]:
parsed = tf.train.Example.FromString(raw_data.numpy())

In [13]:
parsed.features.feature['input_ids']

int64_list {
  value: 2
  value: 7864
  value: 18704
  value: 255
  value: 3684
  value: 12
  value: 19411
  value: 5673
  value: 3325
  value: 26078
  value: 6281
  value: 5659
  value: 15
  value: 8538
  value: 5746
  value: 6972
  value: 16
  value: 13
  value: 544
  value: 5649
  value: 4
  value: 6718
  value: 4
  value: 12
  value: 20
  value: 12884
  value: 4
  value: 13
  value: 18958
  value: 3018
  value: 6871
  value: 4
  value: 32
  value: 14583
  value: 3105
  value: 17853
  value: 4
  value: 7430
  value: 6
  value: 541
  value: 3091
  value: 3273
  value: 5661
  value: 4334
  value: 12
  value: 13930
  value: 6875
  value: 7904
  value: 8109
  value: 4
  value: 15
  value: 8277
  value: 14197
  value: 3113
  value: 13
  value: 6
  value: 5949
  value: 7788
  value: 17
  value: 7720
  value: 5697
  value: 6025
  value: 4
  value: 532
  value: 3018
  value: 3473
  value: 3022
  value: 12
  value: 8088
  value: 6215
  value: 6375
  value: 13595
  value: 3036
  value: 13
  v

In [19]:
train_dataset = tf.data.TFRecordDataset(filenames)
train_dataset = train_dataset.map(_parse_function)


In [14]:

with strategy.scope():

    train_dataset = tf.data.TFRecordDataset(filenames)
    train_dataset = train_dataset.map(_parse_function)
    train_dataset_iters = train_dataset.shuffle(BUFFER_SIZE).batch(1) 
    
    train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset_iters)


## Bert_Config

In [9]:
# sub_model (core_model 필요 Config)

vocab_size = 32000 # 
hidden_size = 768 # Transformer hidden Layers
type_vocab_size = 12 #: The number of types that the 'type_ids' input can take.
num_layers = 12
num_attention_heads = 12
max_seq_length = 256 # 512
dropout_rate = .1
# attention_dropout_rate = .1
inner_dim = 3072
# hidden_act = 'gelu'
initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)

# Pretrain Model 필요 Config
max_predictions_per_seq = 255


## Model Define

In [10]:
with strategy.scope() :

    model, sub_model = models.get_bert_models_fn(vocab_size
                                             , hidden_size
                                             , type_vocab_size
                                             , num_layers
                                             , num_attention_heads
                                             , max_seq_length
                                             , max_predictions_per_seq
                                             , dropout_rate
                                             , inner_dim 
                                             , initializer)

NameError: name 'strategy' is not defined

## Traning Config

ValueError: in user code:

    C:\Users\LGCNS\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\LGCNS\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\LGCNS\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\LGCNS\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\LGCNS\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:541 train_step  **
        self.trainable_variables)
    C:\Users\LGCNS\AppData\Roaming\Python\Python37\si